In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
from matplotlib import pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import seaborn as sns
import cufflinks as cf
cf.set_config_file(offline=False, world_readable=True, theme='ggplot')
from DataFrame import *
from netCDF4 import Dataset as nc, num2date

In [2]:
sns.set_style("whitegrid")
cpolFile = os.path.join(os.environ['HOME'], 'Data', 'Extremes', 'CPOL', 'CPOL_area_avg_precip.pkl')
largeScaleFile = os.path.join(os.environ['HOME'], 'Data', 'Extremes', 'CPOL', 'CPOL-large-scale_forcing.pkl')
popeFile = os.path.join(os.environ['HOME'], 'Data', 'Extremes', 'CPOL', 'Pope_regimes.pkl')

#### Load the datasets for CPOL rainfall and Large-scale forcing

In [3]:
cpolFrame = pd.read_pickle(cpolFile)
cpol6H = pd.read_pickle(largeScaleFile)[['omega-0615hPa','regime','CPOL_rain']]
cpol10m = cpolFrame['all']
#cpol1H = convScale(cpolFrame['all']/6,'sum','1H')
#cpol6H = dict(CpolRain = convScale(cpol1H,'mean','6H'))
#cpol1H = dict(CpolRain =cpol1H)
#cpol6H['Regime'] = convScale(cpolFrame['regime'],'mean','6H')
#cpol1H['Regime'] = convScale(cpolFrame['regime'],'mean','1H')
#cpol6H = pd.DataFrame(cpol6H)
#cpol1H = pd.DataFrame(cpol1H)


### The Timeseries of the 6 hly Cpol Data (Pope Regimes added)

In [4]:
regime = convScale(cpol6H['regime'],'mean','1D')

In [5]:
periods = partition(pd.read_pickle(popeFile))

In [6]:
color = {1. : 'orange', 2. : 'green', 3. : 'red', 4. : 'purple', 5. : 'brown'}

In [7]:
shapes = get_shapes(color,periods)

In [8]:
data = [ go.Scatter(x=cpol6H['CPOL_rain'].index,y=cpol6H['CPOL_rain'].values) ]

In [9]:
annotations = []
yy = 1
for reg, co in color.items():
    annotations.append(
        dict(
            x=1.08,
            y=yy,
            xref='paper',
            yref='paper',
            text='Regime %1.0f'%reg,
            showarrow=False,
            font=dict(
                family='sans',
                size=12,
                color='white'
            ),
            align='center',
            bordercolor='#c7c7c7',
            borderwidth=1,
            borderpad=4,
            bgcolor=co,
            opacity=0.8
        )
    )
    yy -= 0.15

In [10]:
layout = dict(shapes=shapes,yaxis=dict(title='6h avg Rainfall [mm/h]'),annotations=annotations)

In [11]:
py.iplot({'data': data, 'layout': layout}, filename='rain_timeseries')

### The Number of Extreme Events by Quantile

In [12]:
quantiles = [80,85,90,95,99]
extremeCount = []
for quan in quantiles:
    extremeCount.append(cpol6H['CPOL_rain'][cpol6H['CPOL_rain']>cpol6H['CPOL_rain'].quantile(quan/100.)].count())

In [13]:
extremeCount = pd.Series(extremeCount,index=quantiles)

In [14]:
extremeCount.iplot(kind='bar', filename='extreme_Count',yTitle='Number of Events Above Quantile',xTitle='Quantile',
                  title='Number of Events vs. Quantile (6 hly)')

### Extreme Rainfall Events (>90)  per Pope Regime

In [15]:
extreme = cpol6H.loc[cpol6H['CPOL_rain']>cpol6H['CPOL_rain'].quantile(90/100.)]
#extreme_2 = cpol6H.loc[cpol1H['CPOL_Rain']>cpol1H['CPOL_Rain'].quantile(90/100.)]

In [16]:
regCount = []
for reg in range(1,6):
    regCount.append(extreme['CPOL_rain'].loc[extreme['regime']==reg].count())
regCount = pd.Series(regCount,index=range(1,6))
regCount.iplot(kind='bar', filename='reg_Count',yTitle='Number of Events in Rainfall Regime',xTitle='Regime',
                  title='Number of Events vs. Rainfall Regime')

In [17]:
compare_1 = [] 
compare_1.append({
            "type": 'violin',
            "y": extreme['CPOL_rain'].values,
            "name": 'all Regimes',
            "box": {
                "visible": True
            },
            "meanline": {
                "visible": True
            }
        })
for reg in range(1,6):
        compare_1.append({
            "type": 'violin',
            "y": extreme['CPOL_rain'].loc[extreme['regime'] == reg].values,
            "name": "Regime %i" %reg,
            "box": {
                "visible": True
            },
            "meanline": {
                "visible": True
            }
        })
        
    
fig_1 = {
    "data": compare_1,
    "layout" : {
        "title": "Radar rain in Different Rainfall Regimes",
        "yaxis": {
            "zeroline": False,
            "title": "6 Hour Rain-Rate [mm/h]"
            
        }
    }
} 

In [18]:
py.iplot(fig_1, filename='rainfall_violine',validate=False)

### Large-scale variables in Extreme Rainfall Situations
##### The Distribution of $\omega_{600}$ for Rainfall Events Above the 90th Percentile 

In [19]:
largeScales = [e for e in extreme.columns if e.lower() not in ('cpol_rain','regime')]
compare = {}
tt = 1
for var in largeScales:
    compare[var]=[]
    compare[var].append({
            "type": 'violin',
            "y": extreme[var].values,
            "name": 'all Regimes',
            "box": {
                "visible": True
            },
            "meanline": {
                "visible": True
            }
        })
    for reg in range(1,6):
        compare[var].append({
            "type": 'violin',
            "y": extreme[var].loc[extreme['regime'] == reg].values,
            "name": "Regime %i" %reg,
            "box": {
                "visible": True
            },
            "meanline": {
                "visible": True
            }
        })
        
    
fig = {
    "data": compare['omega-0615hPa'],
    "layout" : {
        "title": "$ \\omega_{600} \\text{ in Different Rainfall Regimes}$",
        "yaxis": {
            "zeroline": False,
            "title": "$ \\omega_{600{}}  \\text{ [hPa/h]}$"
            
        }
    }
} 

In [20]:
py.iplot(fig, filename='omega_violine',validate=False)

### Powerlaw of Rainfall aka Self-Organized Crticality
First test if rainfall follows as powerlaw. For this bin the rainfall-rate (0.01 steps) and rank them

In [21]:
cpol1hBins = cpol10m.round(2).dropna()
cpol1hBins = cpol1hBins.loc[cpol1hBins>0]
#cpol1hBins = cpol1hBins.loc[cpol1hBins<10]

In [22]:
unique = np.unique(cpol1hBins.values)

In [23]:
rank = [cpol1hBins.loc[cpol1hBins == i].count() for i in unique]

In [24]:
m, t= -87.54385964912281, 701.3508771929825
x=np.linspace(1,8,100)
y=m*x+t

In [25]:
data = [ go.Scatter(x=unique,y=rank),go.Scatter(x=[x[0],x[-1]],y=[56*y[0],y[-1]])]
layout = dict(yaxis=dict(title='log(Number of Occurrances)',type='log'),
              xaxis=dict(title='log(Rain-rate)',type='log'))
py.iplot({'data': data, 'layout': layout}, filename='rain_rank')